# Day 7

This builds on [day 5](./05.ipynb) 


 

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from aoc.day07 import IntCodeComputer

In [ ]:
# The example program uses an input instruction to ask for a single number. 
# The program will then output 999 if the input value is below 8, output 1000 if 
# the input value is equal to 8, or output 1001 if the input value is greater than 8.
computer = IntCodeComputer([3,21,1008,21,8,20,1005,20,22,107,8,21,20,1006,20,31,
                            1106,0,36,98,0,0,1002,21,125,20,4,20,1105,1,46,104,
                            999,1105,1,46,1101,1000,1,20,4,20,1105,1,46,98,99])
computer.run()

In [ ]:
def computer_chain(memory, inputs, computers=5):
    # Always input 0 on the first run
    outputs = [0]

    def input_value():
        value = inputs.pop(0)
        return value

    def output_value(value):
        return outputs.append(value)   

    def create_computer():
        computer = IntCodeComputer(list(memory), debug=False)
        computer.input_value = input_value
        computer.output_value = output_value
        return computer
    
    for c in range(0, computers):
        inputs.insert(1, outputs.pop())
        create_computer().run()

    
    return outputs

In [ ]:
def find_max(memory):
    import numpy as np
    from collections import Counter
    from tqdm import trange
    max_v = -1
    setting = None
    
    # Each wheel has 5 combinations. Therefore we have 5**5 possible
    for i in trange(0, 5**5):
        v = "{:05}".format(int(np.base_repr(i, 5)))
        v = [int(i) for i in v]
        if max(Counter(v).values()) > 1:
            continue
        o = computer_chain(memory, list(v))[0]
        if o > max_v:
            setting = v
            max_v = o
    
    return max_v, setting

In [ ]:
# 43210
prog = [3,15,3,16,1002,16,10,16,1,16,15,15,4,15,99,0,0]
display(computer_chain(prog, [4,3,2,1,0]))
find_max(prog)

In [ ]:
# 54321
prog = [3,23,3,24,1002,24,10,24,1002,23,-1,23,101,5,23,23,1,24,23,23,4,23,99,0,0]
display(computer_chain(prog, [0,1,2,3,4]))
find_max(prog)

In [ ]:
# 65210
prog = [3,31,3,32,1002,32,10,32,1001,31,-2,31,1007,31,0,33,1002,33,7,33,1,33,31,
        31,1,32,31,31,4,31,99,0,0,0]
display(computer_chain(prog, [1,0,4,3,2]))
find_max(prog)

In [ ]:
with open("07-input.txt", "rt") as FILE:
    data = FILE.read()
    data = data.split(",")
    data = [int(d) for d in data]
data[:8]

In [ ]:
find_max(data)

## Part 2

In [ ]:
def piped_computer_chain(memory, inputs, computers=5):
    def create_computer():
        computer = IntCodeComputer(list(memory),debug=False)

        def input_value():
            if len(computer.in_pipe) == 0:
                return None
            else:
                return computer.in_pipe.pop(0)

        def output_value(value):
            return computer.out_pipe.append(value)  
        
        computer.input_value = input_value
        computer.output_value = output_value
        return computer
    
    comps = []
    for c in range(computers):
        computer = create_computer()
        computer.in_pipe = [inputs[c]]
        comps.append(computer)

    # Hook em all up
    for c in range(computers-1):
        comps[c].out_pipe = comps[c+1].in_pipe
        
    # Create loop
    comps[-1].out_pipe = comps[0].in_pipe 
    
    # Starting value
    comps[0].in_pipe.append(0)

    # Final Output
    output = comps[-1].out_pipe

    while len(comps)>0:
        for c in comps:
            exit = c.run()
            if (exit == -1):
                comps.remove(c)
                
        
    return output

In [ ]:
def piped_find_max(memory):
    import numpy as np
    from collections import Counter
    from tqdm import trange
    max_v = -1
    setting = None
    
    # Each wheel has 5 combinations. Therefore we have 5**5 possible
    for i in trange(5**5):
        v = "{:05}".format(int(np.base_repr(i, 5)))
        v = [5+int(i) for i in v]
        if max(Counter(v).values()) > 1:
            continue
        o = piped_computer_chain(memory, list(v))[0]
        if o > max_v:
            setting = v
            max_v = o
    
    return max_v, setting

In [ ]:
prog = [3,26,1001,26,-4,26,3,27,1002,27,2,27,1,27,26,27,4,27,1001,28,-1,28,1005,28,6,99,0,0,5]
display(piped_computer_chain(prog, [9,8,7,6,5]))
piped_find_max(prog)

In [ ]:
prog = [3,52,1001,52,-5,52,3,53,1,52,56,54,1007,54,5,55,1005,55,26,1001,54,
        -5,54,1105,1,12,1,53,54,53,1008,54,0,55,1001,55,1,55,2,53,55,53,4,
        53,1001,56,-1,56,1005,56,6,99,0,0,0,0,10]
display(piped_computer_chain(prog, [9, 7, 8, 5, 6]))
piped_find_max(prog)

In [ ]:
piped_find_max(data)